In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

  # Install the latest version of torchtext library quietly without showing output
!pip install torchtext -qq

!pip install transformers evaluate wandb datasets accelerate bitsandbytes -U -qq ## NEW LINES ##
basepath = '/content/drive/MyDrive/NLP/'

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 28.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 36.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 39.2 MB/s e

In [ ]:
!pip install accelerate -qq

In [ ]:
from pathlib import Path
# wandb library
import wandb
import warnings
warnings.simplefilter('ignore')
from tqdm import tqdm
from sklearn import metrics
import torch
torch.cuda.empty_cache()
#from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)
# Importing stock ml libraries
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd

from sklearn import metrics
import transformers

from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)
import os, sys
os.environ['TOKENIZERS_PARALLELISM']='false'

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GemmaTokenizer
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

import transformers
from transformers import DataCollatorForSeq2Seq, AutoTokenizer, RobertaForSequenceClassification,AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer, set_seed
from transformers import T5ForSequenceClassification, AutoTokenizer
from transformers import AutoConfig


from transformers import AutoTokenizer, AutoModelForCausalLM
import datasets
from datasets import Dataset, DatasetDict

import evaluate

!pip install peft -qq

!git lfs install

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.4 MB/s eta 0:00:00
Git LFS initialized.


In [ ]:
from peft import (
    TaskType,
    LoraConfig,
    get_peft_model,
)

In [ ]:
base_folder = Path(basepath)
data_folder = base_folder/'Homework/Homework7'
model_folder = base_folder/'models/nlp_spring_2024/ed/nn'
custom_functions = base_folder/'custom-functions'

In [ ]:
sys.path.append('/content/drive/MyDrive/NLP/custom-functions')
import CustomPreprocessorSpacy as cp

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'


In [ ]:
train_data = pd.read_csv(data_folder/ 'train.csv')
test= pd.read_csv(data_folder/ 'test.csv')

In [ ]:
train_data.drop(columns=["ID"],inplace=True)
y_test = test.drop(['ID','Tweet'],axis=1)
for column in y_test:
    y_test[column] = y_test[column].apply(lambda x: 0 if x == "NONE" else 1)
y_test = y_test.apply(lambda x: list(map(int, x)))
test_data = pd.concat([test['Tweet'],y_test],axis=1)

train_data.head()


,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,“Worry is a down payment on a problem you may ...,0,1,0,0,0,0,1,0,0,0,1
1,Whatever you decide to do make sure it makes y...,0,0,0,0,1,1,1,0,0,0,0
2,@Max_Kellerman it also helps that the majorit...,1,0,1,0,1,0,1,0,0,0,0
3,Accept the challenges so that you can literall...,0,0,0,0,1,0,1,0,0,0,0
4,My roommate: it's okay that we can't spell bec...,1,0,1,0,0,0,0,0,0,0,0


In [ ]:
test_data.head()

,Tweet,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,@Adnan__786__ @AsYouNotWish Dont worry Indian ...,0,0,0,0,0,0,0,0,0,0,0
1,"Academy of Sciences, eschews the normally sobe...",0,0,0,0,0,0,0,0,0,0,0
2,I blew that opportunity -__- #mad,0,0,0,0,0,0,0,0,0,0,0
3,This time in 2 weeks I will be 30... 😥,0,0,0,0,0,0,0,0,0,0,0
4,#Deppression is real. Partners w/ #depressed p...,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
labels = [x for x in train_data.columns if x not in ['Tweet']]
id2label = {idx:label for idx, label in enumerate(labels)}

label2id = {label:idx for idx, label in enumerate(labels)}
NUM_OF_LABELS = len(labels)

In [ ]:
labels = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']

# Creating a new DataFrame with the 'tweet' column followed by the emotions columns as a list
train_df = pd.DataFrame({'text': train_data['Tweet'], 'label': train_data[labels].values.tolist()})
test_df = pd.DataFrame({'text': test_data['Tweet'], 'label': test_data[labels].values.tolist()})
train_df.head()

,text,label
0,“Worry is a down payment on a problem you may ...,"[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1]"
1,Whatever you decide to do make sure it makes y...,"[0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0]"
2,@Max_Kellerman it also helps that the majorit...,"[1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0]"
3,Accept the challenges so that you can literall...,"[0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]"
4,My roommate: it's okay that we can't spell bec...,"[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)
train_testEval = train_dataset.train_test_split(train_size=0.80)
val_eval = train_testEval['test']

ds = DatasetDict({
    'train' : train_testEval['train'],
    'val' : val_eval,
    'test':test_dataset

})

print("Training Dataset Shape:", ds['train'].shape)
print("Evaluation Dataset Shape:", ds['val'].shape)
print("Testing Dataset Shape",ds['test'].shape)
ds['train'][25]

Training Dataset Shape: (6179, 2)
Evaluation Dataset Shape: (1545, 2)
Testing Dataset Shape (3259, 2)


{'text': "@serendipity127_ @zombiecalorie @Angel_Eyes66 I'm sensing a theme here tho lol ",
 'label': [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0]}

In [ ]:
checkpoint = "google/gemma-1.1-2b-it"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [ ]:
def tokenize_fn(batch):
    return tokenizer(text = batch["text"], truncation=True)

In [ ]:
tokenized_dataset= ds.map(tokenize_fn, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(
    ['text']
)
# tokenized_dataset.set_format(type='torch')

Map:   0%|          | 0/6179 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1545 [00:00<?, ? examples/s]

Map:   0%|          | 0/3259 [00:00<?, ? examples/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 6179
    })
    val: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 1545
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 3259
    })
})

In [ ]:
accuracy_metric = evaluate.load('accuracy', 'multilabel')
f1 = evaluate.load('f1','multilabel')

def compute_metrics(eval_pred):
    logits, labels = eval_pred

    #logits= logits[0]

    # print(logits.shape)
    preds = (logits > 0).astype(int)

    accuracy = accuracy_metric.compute(predictions=preds, references=labels)
    f1_micro = f1.compute(predictions=preds, references=labels, average='micro')
    f1_macro = f1.compute(predictions=preds, references=labels, average='macro')
    return {'f1_micro':f1_micro['f1'],
            'f1_macro':f1_macro['f1'],
            'accuracy':accuracy['accuracy'],
            }

In [ ]:
# Define the directory where model checkpoints will be saved
run_name = "emotions_gemma_lora_im"
base_folder = Path(basepath)
model_folder = base_folder / "models"/run_name
# Create the directory if it doesn't exist
model_folder.mkdir(exist_ok=True, parents=True)

# Configure training parameters
training_args = TrainingArguments(
    # Training-specific configurations
    num_train_epochs=10,  # Total number of training epochs
    # Number of samples per training batch for each device
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # gradient_accumulation_steps=8,

    weight_decay=0.1,  # Apply L2 regularization to prevent overfitting
    learning_rate=1e-4,  # Step size for the optimizer during training
    lr_scheduler_type='linear',
    warmup_steps=0,  # Number of warmup steps for the learning rate scheduler
    optim='adamw_torch',  # Optimizer,
    max_grad_norm = 1.0,

    # Checkpoint saving and model evaluation settings
    output_dir=str(model_folder),  # Directory to save model checkpoints
    evaluation_strategy='steps',  # Evaluate model at specified step intervals
    eval_steps=100,  # Perform evaluation every 10 training steps
    save_strategy="steps",  # Save model checkpoint at specified step intervals
    save_steps=200,  # Save a model checkpoint every 10 training steps
    load_best_model_at_end=True,  # Reload the best model at the end of training
    save_total_limit=1,  # Retain only the best and the most recent model checkpoints
    # Use 'accuracy' as the metric to determine the best model
    metric_for_best_model="eval_f1_macro",
    greater_is_better=True,  # A model is 'better' if its accuracy is higher


    # Experiment logging configurations (commented out in this example)
    logging_strategy='steps',
    logging_steps=200,
    report_to='wandb',  # Log metrics and results to Weights & Biases platform
    run_name=run_name,  # Experiment name for Weights & Biases

    # fp16=False,
    # bf16=False,
    # tf32= False
)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels=11,
                                                           problem_type="multi_label_classification" )

config = AutoConfig.from_pretrained(checkpoint)
id2label= {id_: label_ for id_, label_ in enumerate(labels)}
label2id = {label_: id_ for id_, label_ in enumerate(labels)}
config.id2label = id2label
config.label2id = label2id
model.config = config


config.json:   0%|          | 0.00/618 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at google/gemma-1.1-2b-it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

GemmaForSequenceClassification(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    

In [ ]:
import re
model_modules = str(model.modules)
pattern = r'\((\w+)\): Linear'
linear_layer_names = re.findall(pattern, model_modules)

names = []
# Print the names of the Linear layers
for name in linear_layer_names:
    names.append(name)
target_modules = list(set(names))
target_modules

['q_proj',
 'o_proj',
 'k_proj',
 'down_proj',
 'up_proj',
 'gate_proj',
 'score',
 'v_proj']

In [ ]:
gemma_peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=100,
    lora_alpha=200,
    lora_dropout=0.01,
    bias="lora_only",
    modules_to_save = ['out_proj', 'dense'],
    target_modules = ['v_proj','up_proj','k_proj','o_proj','q_proj','score','down_proj','gate_proj']
)
gemma_peft_model = get_peft_model(model, gemma_peft_config )
gemma_peft_model.print_trainable_parameters()

trainable params: 122,801,228 || all params: 2,629,202,072 || trainable%: 4.670665267907183


In [ ]:
gemma_peft_config.target_modules

{'down_proj',
 'gate_proj',
 'k_proj',
 'o_proj',
 'q_proj',
 'score',
 'up_proj',
 'v_proj'}

In [ ]:
gemma_peft_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GemmaForSequenceClassification(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.01, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=100, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=100, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
   

In [ ]:
def calculate_pos_weights(dataset):
    # Initialize counters for all labels
    num_labels = len(dataset['train']['label'][0])
    total_positives = [0] * num_labels
    total_negatives = [0] * num_labels

    # Count positives and negatives for each label
    for label_array in dataset['train']['label']:
        for i, label in enumerate(label_array):
            if label == 1:
                total_positives[i] += 1
            else:
                total_negatives[i] += 1

    # Calculate pos_weight for each label
    pos_weight = [total_negatives[i] / max(total_positives[i], 1) for i in range(num_labels)]
    return torch.tensor(pos_weight)

# Calculate the pos_weight using the training set
pos_weights = calculate_pos_weights(ds)


In [ ]:
pos_weights

tensor([ 1.6994,  6.1187,  1.6361,  4.5318,  1.6726,  8.2500,  2.4007,  7.6906,
         2.4195, 19.6656, 18.5538])

In [ ]:
pos_weights= torch.tensor([2., 3., 2., 2., 2., 3., 2., 3., 2., 4., 4.])

In [ ]:
import torch.nn as nn
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels").float()  # Ensure labels are float for BCE loss
        outputs = model(**inputs)
        logits = outputs.get("logits")

        device = next(model.parameters()).device

        loss_fct = nn.BCEWithLogitsLoss(pos_weight=pos_weights.to(device))
        loss = loss_fct(logits, labels)

        return (loss, outputs) if return_outputs else loss


In [ ]:
trainer = CustomTrainer(
    model=gemma_peft_model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["val"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

In [ ]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
trainer.train()  # start training

Step,Training Loss,Validation Loss,F1 Micro,F1 Macro,Accuracy
100,No log,0.746433,0.612093,0.484045,0.148867
200,0.301900,0.705918,0.642896,0.521265,0.192233
300,0.301900,0.558022,0.668653,0.552452,0.216181
400,0.564800,0.593330,0.658507,0.511064,0.204531
500,0.564800,0.544573,0.675297,0.556753,0.219417
600,0.531800,0.512414,0.696672,0.585229,0.242071
700,0.531800,0.523750,0.671502,0.583556,0.205825
800,0.499900,0.526775,0.676498,0.588475,0.204531
900,0.499900,0.572183,0.669511,0.575662,0.183819
1000,0.367500,0.620625,0.676882,0.559268,0.207767


TrainOutput(global_step=7730, training_loss=0.1293316110130038, metrics={'train_runtime': 9123.2277, 'train_samples_per_second': 6.773, 'train_steps_per_second': 0.847, 'total_flos': 3.0590720418204144e+16, 'train_loss': 0.1293316110130038, 'epoch': 10.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.5267751216888428,
 'eval_f1_micro': 0.6764976958525345,
 'eval_f1_macro': 0.5884750442135105,
 'eval_accuracy': 0.2045307443365696,
 'eval_runtime': 44.8558,
 'eval_samples_per_second': 34.444,
 'eval_steps_per_second': 4.325,
 'epoch': 10.0}

In [ ]:
best_model_checkpoint_step = trainer.state.best_model_checkpoint.split('-')[-1]
print(f"The best model was saved at step {best_model_checkpoint_step}.")

The best model was saved at step 800.


In [ ]:
model_folder

PosixPath('/content/drive/MyDrive/NLP/models/emotions_gemma_lora_im')

In [ ]:
model_checkpoint = str(model_folder) + '/checkpoint-' + best_model_checkpoint_step

In [ ]:
valid_results = trainer.evaluate(tokenized_dataset["val"])

In [ ]:
valid_results

{'eval_loss': 0.5267751216888428,
 'eval_f1_micro': 0.6764976958525345,
 'eval_f1_macro': 0.5884750442135105,
 'eval_accuracy': 0.2045307443365696,
 'eval_runtime': 44.198,
 'eval_samples_per_second': 34.956,
 'eval_steps_per_second': 4.389,
 'epoch': 10.0}

In [ ]:
gemma_peft_model.save_pretrained('/content/drive/MyDrive/NLP/testing')

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
gemma_peft_model.push_to_hub("zura1101/gemma_peft_model_emotion_detection")

adapter_model.safetensors:   0%|          | 0.00/493M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/zura1101/gemma_peft_model_emotion_detection/commit/56116fcaf10d09965fc82b13f8f7c9cd2f933bab', commit_message='Upload model', commit_description='', oid='56116fcaf10d09965fc82b13f8f7c9cd2f933bab', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSequenceClassification, AutoTokenizer

peft_model_id = "zura1101/gemma_peft_model_emotion_detection"
config = PeftConfig.from_pretrained(peft_model_id)
config.base_model_name_or_path
base_model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path,
                                                                num_labels=11,
                                                                problem_type="multi_label_classification")
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)


adapter_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of GemmaForSequenceClassification were not initialized from the model checkpoint at google/gemma-1.1-2b-it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Load the Lora model
inference_model = PeftModel.from_pretrained(base_model, peft_model_id)

adapter_model.safetensors:   0%|          | 0.00/493M [00:00<?, ?B/s]

In [ ]:
inference_model

PeftModelForSequenceClassification(
  (base_model): LoraModel(
    (model): GemmaForSequenceClassification(
      (model): GemmaModel(
        (embed_tokens): Embedding(256000, 2048, padding_idx=0)
        (layers): ModuleList(
          (0-17): 18 x GemmaDecoderLayer(
            (self_attn): GemmaSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.01, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=100, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=100, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear(
   

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_eval_batch_size=8,
    do_train=False,
    do_eval=True,
    report_to=[]
)

In [ ]:
trainer = CustomTrainer(
    model=inference_model,
    args=training_args,
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,

)

In [ ]:
test_results = trainer.predict(tokenized_dataset["test"])

# The predictions object contains various fields including predictions
logits = test_results.predictions

# Apply sigmoid to get the probabilities
probs = torch.sigmoid(torch.Tensor(logits))

# Apply a threshold to get the binary predictions
threshold = 0.5
y_pred = (probs >= threshold).int().numpy()

In [ ]:
y_pred

array([[1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [1, 0, 1, ..., 1, 0, 0],
       ...,
       [1, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [ ]:
y_pred_df = pd.DataFrame(y_pred,columns=labels)
output_submission = pd.concat([test['ID'],y_pred_df],axis=  1)
output_submission

,ID,anger,anticipation,disgust,fear,joy,love,optimism,pessimism,sadness,surprise,trust
0,2018-01559,1,0,1,1,0,0,1,0,0,0,0
1,2018-03739,0,0,0,0,0,0,0,0,1,0,0
2,2018-00385,1,0,1,0,0,0,0,0,1,0,0
3,2018-03001,0,1,0,1,0,0,0,1,1,0,0
4,2018-01988,0,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3254,2018-03848,1,0,1,0,0,0,0,0,0,0,0
3255,2018-00416,0,1,0,1,0,0,0,0,0,0,1
3256,2018-03717,1,0,1,0,1,0,0,0,0,0,0
3257,2018-03504,0,0,0,0,1,1,1,0,0,0,0


In [ ]:
output_submission.to_csv('/content/drive/MyDrive/NLP/output_submission.csv', index=False)

In [ ]:
wandb.finish()

eval/accuracy,▁▄▇▇▇▆▇▇▅▅▇▅▆▆▆▇▇▇▇▆▇█▇▇█▇▇▇▇██████████▇
eval/f1_macro,▁▅▇▇██▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█
eval/f1_micro,▁▅████▇█▇▇█▇█▇▇▇▇▇▇▇████████████████████
eval/loss,▅▂▁▁▁▁▂▁▂▃▃▃▃▄▄▄▅▅▅▅▆▆▆▆▆▇▆▇▇▇▇████████▁
eval/runtime,▁▆▇▆▆▆▆▆▆▆▅▆▇▇▆▅▆▅▆▆▇▇█▇▇▇███▇▇█████▇▇█▆
eval/samples_per_second,█▃▂▃▃▃▃▃▃▃▄▃▂▂▃▃▃▄▃▃▂▂▁▂▂▂▁▁▁▂▂▁▁▁▁▁▂▂▁▃
eval/steps_per_second,█▃▂▃▃▃▃▃▃▃▄▃▂▂▃▃▃▄▃▃▂▂▁▂▂▂▁▁▁▂▂▁▁▁▁▁▂▂▁▃
train/epoch,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇████
train/grad_norm,█▅▆▆▃▄▅▃▃▂▅▅▃▂▄▃▂▂▃▁▄▃▁▁▁▁▁▂▁▂▁▂▁▁▁▁▁▁▁▁
train/learning_rate,███████▇▇▇▇▇▇▆▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁
